In [3]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from math import nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    return BeautifulSoup(req.text,"html.parser")

def get_max(page):
    pagination = page.find("div",id="pagination")
    numbers = []
    for tag in pagination.find_all("a",class_="page-numbers"):
        try:
            number = int(tag.get_text())
            numbers.append(number)
        except: pass
    max_page = max(numbers)
    return max_page

def get_jobs(url,include=None):
    jobs = []
    page = render_html(url)
    for a in page.find_all("a",class_=["job-block","d-block"]):
        try:
            try: link = a['data-url']
            except: link = nan
            try: title = a['data-title']
            except: title = nan
            try: city = a['data-location']
            except: city = nan
            try: wage = a.find("p").find_next("p").get_text()
            except: wage = nan
            try: hours = a['data-hours']
            except: hours = nan
            try: type_ = a['data-industry']
            except: type_ = nan
            job = {
                "Link": link,
                "Title": title,
                "City": city,
                "Wage": wage,
                "Hours": hours,
                "Type": type_
            }
            jobs.append(job)
        except: pass
    if include == "max_page":
        max_page = get_max(page)
        return jobs, max_page
    else:
        return jobs

In [4]:
# url = "https://www.pran.pl/oferty-pracy/?_page=3"
# page = render_html(url)

In [5]:
url = "https://www.pran.pl/oferty-pracy/?_page=1"
jobs, max_page = get_jobs(url,include="max_page")
for i in range(2,max_page+1):
    url = f"https://www.pran.pl/oferty-pracy/?_page={i}"
    new_jobs = get_jobs(url)
    jobs.extend(new_jobs)
    print(f"{i}/{len(jobs)}",end=",")
print("\b.")

2/18,3/27,4/36,5/45,6/54,7/63,8/72,9/81,10/90,11/99,12/108,13/117,14/126,15/135,16/144,17/152.


In [6]:
import pandas as pd

df = pd.DataFrame(jobs)
df = df.dropna()
df.to_excel("pran.xlsx",index=False,encoding="utf-8")